In [2]:
import getEPH
import functionsForModels
import make_dummy
import schoolYears
import categorize
import functionsForModels
import createVariables

import pandas as pd
#http://statsmodels.sourceforge.net/devel/examples/generated/example_wls.html
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
np.random.seed(1024)
%matplotlib inline

/Users/IlanReinstein/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
getEPH.getEPHdbf('t310')

('Downloading', 't310')
file in place, creating CSV file
csv file cleanDataHousehold t310 .csv successfully created in folder data/
csv file cleanData t310 .csv successfully created in folder data/


In [3]:
data1 = pd.read_csv('data/cleanDatat310.csv')
data2 = categorize.categorize(data1)
data3 = schoolYears.schoolYears(data2)
data4 = createVariables.createVariables(data3)


In [4]:
jobsAndIncome = (data4.activity == 1) & (data4.P21 > 1) 
headAndSpouse = (data4.familyRelation == 1)|(data4.familyRelation == 2)
data = data4.copy().loc[jobsAndIncome,:]


In [5]:
#data['job'] = (data.activity==1).astype(int)

In [6]:
variablesOfInterest = ['age',
                              'age2',
                              'female',
                              'education',
                              'education2']

In [7]:
model = functionsForModels.runModel(dataset = data, income = 'lnIncome', variables = variablesOfInterest)

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.287
Model:                            WLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     289.9
Date:                Wed, 30 Nov 2016   Prob (F-statistic):          2.70e-261
Time:                        11:51:08   Log-Likelihood:                -4091.5
No. Observations:                3608   AIC:                             8195.
Df Residuals:                    3602   BIC:                             8232.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.5938      0.138     40.537      0.0

In [8]:
X = sm.add_constant(data.copy().loc[:,variablesOfInterest].values)

In [9]:
data['predictedLnIncome'] = model.predict(X)

In [10]:
data.head()

,CODUSU,NRO_HOGAR,COMPONENTE,AGLOMERADO,PONDERA,familyRelation,female,age,schoolLevel,finishedYear,...,primary,secondary,university,education,education2,age2,id,lnIncome,lnIncomeT,predictedLnIncome
2,307861,1,1,32,1674,1,0,42,2.0,1,...,7.0,0.0,0.0,7.0,49.0,1764,3078611,8.006368,8.006368,7.478150
3,307861,1,2,32,1674,2,1,44,7.0,1,...,7.0,5.0,5.0,17.0,289.0,1936,3078611,7.937375,7.937375,7.991038
7,308278,1,1,32,1320,1,0,38,8.0,1,...,7.0,5.0,5.0,17.0,289.0,1444,3082781,8.517193,8.699515,8.485189
8,308278,1,2,32,1320,2,1,28,7.0,1,...,7.0,5.0,5.0,17.0,289.0,784,3082781,8.294050,8.294050,7.750667
9,311937,1,1,32,1281,1,0,63,7.0,1,...,7.0,5.0,5.0,17.0,289.0,3969,3119371,8.242756,8.242756,8.336136


In [11]:
cleanData = data.copy().loc[((jobsAndIncome) & (headAndSpouse)) ,
                              ['id',
                            'familyRelation',
                              'age',
                              'age2',
                              'female',
                              'education',
                              'education2',
                              'primary',
                              'secondary',
                              'university',
                              'P21',
                              'P47T',
                              'lnIncome',
                               u'lnIncomeT',
                              'predictedLnIncome']]

In [12]:
pivot = cleanData.pivot(index='id', columns='familyRelation')
pivot.head()

age          age2        female      education        \
familyRelation     1     2       1      2      1    2         1     2   
id                                                                      
1250971         57.0   NaN  3249.0    NaN    1.0  NaN      14.0   NaN   
1256691         61.0   NaN  3721.0    NaN    0.0  NaN       7.0   NaN   
1256892         31.0  23.0   961.0  529.0    0.0  1.0      12.0  12.0   
1261181         38.0  30.0  1444.0  900.0    0.0  1.0       7.0   7.0   
1261521         25.0  25.0   625.0  625.0    1.0  0.0      16.0  12.0   

               education2           ...        P21            P47T          \
familyRelation          1      2    ...          1       2       1       2   
id                                  ...                                      
1250971             196.0    NaN    ...     2800.0     NaN  4200.0     NaN   
1256691              49.0    NaN    ...     2500.0     NaN  6500.0     NaN   
1256892             144.0  144.0    ...     2200.0   500.0  2200.0   900.0   
1261181              49.0   49.0    ...     1200.0   240.0  1200.0  1195.0   
1261521             256.0  144.0    ...     2000.0  2000.0  2000.0  2900.0   

                lnIncome           lnIncomeT           predictedLnIncome  \
familyRelation         1         2         1         2                 1   
id                                                                         
1250971         7.937375       NaN  8.342840       NaN          7.524758   
1256691         7.824046       NaN  8.779557       NaN          7.337371   
1256892         7.696213  6.214608  7.696213  6.802395          7.752868   
1261181         7.090077  5.480639  7.090077  7.085901          7.441214   
1261521         7.600902  7.600902  7.600902  7.972466          7.527562   

                          
familyRelation         2  
id                        
1250971              NaN  
1256691              NaN  
1256892         6.987607  
1261181         6.756999  
1261521         7.593261  

[5 rows x 26 columns]

In [13]:
pivot.columns

MultiIndex(levels=[[u'age', u'age2', u'female', u'education', u'education2', u'primary', u'secondary', u'university', u'P21', u'P47T', u'lnIncome', u'lnIncomeT', u'predictedLnIncome'], [1, 2]],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=[None, u'familyRelation'])

In [14]:
pivot.to_csv('data/pivotInd.csv')

In [20]:
#para leer en otro archivo
dataN = pd.read_csv('data/pivotInd.csv', names = ['id','headAge','spouseAge','headAge2','spouseAge2',
                                                  'headFemale','spouseFemale','headEduc','spouseEduc',
                                                  'headEduc2','spouseEduc2','headPrimary','spousePrimary',
                                                  'headSecondary','spouseSecondary','headUniversity','spouseUniversity',
                                                  'headP21','spouseP21','headP47T','spouseP47T',
                                                  'headLnIncome','spouseLnIncome','headLnIncomeT','spouseLnIncomeT',
                                                  'headPredictedLnIncome','spousePredictedLnIncome'
                                                 ],skiprows = 3)
np.where(np.isnan(pivot.values),0,pivot.values)


array([[   57.        ,     0.        ,  3249.        , ...,
            0.        ,     7.52475804,     0.        ],
       [   61.        ,     0.        ,  3721.        , ...,
            0.        ,     7.33737128,     0.        ],
       [   31.        ,    23.        ,   961.        , ...,
            6.80239476,     7.75286813,     6.98760733],
       ..., 
       [   43.        ,     0.        ,  1849.        , ...,
            0.        ,     7.37490608,     0.        ],
       [    0.        ,    66.        ,     0.        , ...,
            7.47306909,     0.        ,     6.6726158 ],
       [   28.        ,     0.        ,   784.        , ...,
            0.        ,     6.70669148,     0.        ]])

In [21]:
dataN = dataN.dropna(axis = 0)
dataN.head()


,id,headAge,spouseAge,headAge2,spouseAge2,headFemale,spouseFemale,headEduc,spouseEduc,headEduc2,...,headP21,spouseP21,headP47T,spouseP47T,headLnIncome,spouseLnIncome,headLnIncomeT,spouseLnIncomeT,headPredictedLnIncome,spousePredictedLnIncome
2,1256892,31.0,23.0,961.0,529.0,0.0,1.0,12.0,12.0,144.0,...,2200.0,500.0,2200.0,900.0,7.696213,6.214608,7.696213,6.802395,7.752868,6.987607
3,1261181,38.0,30.0,1444.0,900.0,0.0,1.0,7.0,7.0,49.0,...,1200.0,240.0,1200.0,1195.0,7.090077,5.480639,7.090077,7.085901,7.441214,6.756999
4,1261521,25.0,25.0,625.0,625.0,1.0,0.0,16.0,12.0,256.0,...,2000.0,2000.0,2000.0,2900.0,7.600902,7.600902,7.600902,7.972466,7.527562,7.593261
5,1261591,47.0,43.0,2209.0,1849.0,0.0,1.0,17.0,17.0,289.0,...,10000.0,9500.0,16000.0,9500.0,9.210340,9.159047,9.680344,9.159047,8.535733,7.986869
14,1282641,42.0,38.0,1764.0,1444.0,0.0,1.0,15.0,15.0,225.0,...,4600.0,3600.0,6900.0,5400.0,8.433812,8.188689,8.839277,8.594154,8.255747,7.677937


In [17]:
pivot.shape

(1916, 26)